In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [67]:
data = pd.read_csv('C:/Users/Windows10/Desktop/코딩/data/jena_climate_2009_2016.csv')

data

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [95]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
p (mbar),420551.0,989.212776,8.358481,913.60,984.20,989.58,994.72,1015.35
T (degC),420551.0,9.450147,8.423365,-23.01,3.36,9.42,15.47,37.28
Tpot (K),420551.0,283.492743,8.504471,250.60,277.43,283.47,289.53,311.34
Tdew (degC),420551.0,4.955854,6.730674,-25.01,0.24,5.22,10.07,23.11
rh (%),420551.0,76.008259,16.476175,12.95,65.21,79.30,89.40,100.00
VPmax (mbar),420551.0,13.576251,7.739020,0.95,7.78,11.82,17.60,63.77
VPact (mbar),420551.0,9.533756,4.184164,0.79,6.21,8.86,12.35,28.32
VPdef (mbar),420551.0,4.042412,4.896851,0.00,0.87,2.19,5.30,46.01
sh (g/kg),420551.0,6.022408,2.656139,0.50,3.92,5.59,7.80,18.13
H2OC (mmol/mol),420551.0,9.640223,4.235395,0.80,6.29,8.96,12.49,28.82


In [97]:
wv = data['wv (m/s)']
max_wv = data['max. wv (m/s)']

bad_wv = wv == -9999.0
bad_max_wv = max_wv == -9999.0

wv[bad_wv] = 0.0
max_wv[bad_max_wv] = 0.0

data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
p (mbar),420551.0,989.212776,8.358481,913.60,984.20,989.58,994.72,1015.35
T (degC),420551.0,9.450147,8.423365,-23.01,3.36,9.42,15.47,37.28
Tpot (K),420551.0,283.492743,8.504471,250.60,277.43,283.47,289.53,311.34
Tdew (degC),420551.0,4.955854,6.730674,-25.01,0.24,5.22,10.07,23.11
rh (%),420551.0,76.008259,16.476175,12.95,65.21,79.30,89.40,100.00
VPmax (mbar),420551.0,13.576251,7.739020,0.95,7.78,11.82,17.60,63.77
VPact (mbar),420551.0,9.533756,4.184164,0.79,6.21,8.86,12.35,28.32
VPdef (mbar),420551.0,4.042412,4.896851,0.00,0.87,2.19,5.30,46.01
sh (g/kg),420551.0,6.022408,2.656139,0.50,3.92,5.59,7.80,18.13
H2OC (mmol/mol),420551.0,9.640223,4.235395,0.80,6.29,8.96,12.49,28.82


In [98]:
data = (data - data[:200000].mean()) / data[:200000].std()

In [99]:
def generator(data, lookback, delay, min_index, max_index, shuffle = False, batch_size = 128, step = 6):
    if max_index is None:
        max_index = len(data) - delay -1
    i = min_index + lookback # 시작점

    while 1:
        if shuffle:
            rows = np.random.randint(i, max_index, size = batch_size) # 시작부터 끝까지의 행 번호를 섞겠다는 뜻
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index)) # 시작부터 끝까지의 행 번호를 섞지 않고 순서대로 배열함
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [100]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(data, lookback= lookback, delay = delay, min_index= 0, max_index= 200000, shuffle = True, step = step, batch_size= batch_size)
val_gen = generator(data, lookback= lookback, delay = delay, min_index= 200001, max_index= 300000, step = step, batch_size= batch_size)
test_gen = generator(data, lookback= lookback, delay = delay, min_index= 300001, max_index= None, step = step, batch_size= batch_size)

val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(data) - 300001 - lookback) // batch_size

In [101]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32, dropout= 0.1, recurrent_dropout= 0.5, return_sequences= True, input_shape = (None, data.shape[-1])),
    tf.keras.layers.GRU(64, activation = 'relu', dropout= 0.1, recurrent_dropout= 0.5),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer = tf.keras.optimizers.RMSprop(), loss = 'mae')

history = model.fit_generator(train_gen, steps_per_epoch= 500, epochs = 40, validation_data= val_gen, validation_steps= val_steps)

C:\Users\Windows10\anaconda3\envs\tf_2.0\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


KeyError: "None of [Int64Index([165764, 165770, 165776, 165782, 165788, 165794, 165800, 165806,\n            165812, 165818,\n            ...\n            167144, 167150, 167156, 167162, 167168, 167174, 167180, 167186,\n            167192, 167198],\n           dtype='int64', length=240)] are in the [columns]"